<a href="https://colab.research.google.com/github/JammieBurger/imlo-project-actual/blob/main/IMLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Initialising Training and Testing Data.

In [27]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets,transforms
from torchvision.transforms import ToTensor
import torch.nn.functional as F

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
trainingData = datasets.Flowers102(
    root="data",
    split='train',
    transform=transform,
    download=True)
testData = datasets.Flowers102(
    root="data",
    split="test",
    download=True,
    transform=transform)

Creating Neural Network

In [ ]:
batchSize = 128

trainDataloader = DataLoader(trainingData,batchSize)
testDataloader = DataLoader(testData,batchSize)
device = ("cuda"if torch.cuda.is_available()
                else "mps"
                if torch.backends.mps.is_available()
                else "cpu")
print(f"Using {device} device")
class NeuralNetwork(nn.Module):
    def __init__(self,inputFeatures=224*224*3,layerOne=2,layerTwo=3,outputFeatures=512):
        super().__init__()
        self.flatten = nn.Flatten()
        self.firstLayer = nn.Linear(inputFeatures,layerOne)
        self.secondLayer = nn.Linear(layerOne,layerTwo)
        self.finalLayer = nn.Linear(layerTwo,outputFeatures)

    def forward(self, x):
        x = self.flatten(x)
        x = F.relu(self.firstLayer(x))
        x = F.relu(self.secondLayer(x))
        x = self.finalLayer(x)
        return x